# Neural Networks

We will use a neural network for creating a model which be able to classify correctly.
As the dataset is small, 400 instances, we will follow a straight-line approach:
- We will use a shallow networks because deep ones need more examples.
- We will use **L2 Regularization (Weight Decay)** and **Early Stopping** to stop training the moment the model stops improving
- A small preprocessing, so neural networks require scaling to converge

## Data preprocessing
Neural networks cannot handle raw categorical text or unscaled numbers effectively.

In [1]:
## Data preprocessing